In [ ]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": " ",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/2tenant-id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://landing-zone-2@ecomadlsns.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/ecom",
extra_configs = configs)

True

In [ ]:
%fs ls '/mnt/ecom/'

path,name,size,modificationTime
dbfs:/mnt/ecom/buyers-raw-2/,buyers-raw-2/,0,1712791747000
dbfs:/mnt/ecom/countries-raw-2/,countries-raw-2/,0,1712791779000
dbfs:/mnt/ecom/sellers-raw-2/,sellers-raw-2/,0,1712791757000
dbfs:/mnt/ecom/users-raw-2/,users-raw-2/,0,1712791769000


In [ ]:
%fs ls '/mnt/ecom/buyers-raw-2/'

path,name,size,modificationTime
dbfs:/mnt/ecom/buyers-raw-2/Buyers-repartition-by-country.parquet,Buyers-repartition-by-country.parquet,12018,1713031741000


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.appName("EcomDataPipeline").getOrCreate()

In [ ]:
spark

In [ ]:
# Read parquet file from /mnt/ecomdata1/user-raw-2 folder
userDF = spark.read.format("parquet")\
    .option("header",'true')\
    .option("inferSchema",'true')\
    .load("/mnt/ecomdata1/users-raw-2")

In [ ]:
userDF.show()

+--------------+----+-----------+-----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+-----------------+---------+-----------------+----------------+----------------+
|identifierHash|type|countryCode|    country|language|socialNbFollowers|socialNbFollows|socialProductsLiked|productsListed|productsSold|productsPassRate|productsWished|productsBought|gender|civilityGenderId|civilityTitle|hasProfilePicture|seniority|seniorityAsMonths|seniorityAsYears|websiteLongevity|
+--------------+----+-----------+-----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+-----------------+---------+-----------------+----------------+----------------+
|    1920991140|user|         us| Etats-Unis|      en|                3|              8|      

In [ ]:
userDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/users")

In [ ]:
buyersDF=spark.read.format("parquet")\
    .option("Header",'true')\
    .option("InferSchema",'true')\
    .load("/mnt/ecom/buyers-raw-2")

In [ ]:
buyersDF.write.format("Delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/buyers")

In [ ]:
countriesDF=spark.read.format("parquet")\
    .option("Header",'true')\
    .option("InferSchema",'true')\
    .load("/mnt/ecom/countries-raw-2")

In [ ]:
countriesDF.write.format("Delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/countires")

In [ ]:
sellersDF=spark.read.format("parquet")\
    .option("Header",'true')\
    .option("InferSchema",'true')\
    .load("/mnt/ecom/sellers-raw-2")

In [ ]:
sellersDF.write.format("Delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/sellers")

In [ ]:
sellersDF.show(5)

+---------+------+---------+----------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+-----------+-----------+------------------+-------------+-----------+-----------------+-----------------+------------------+
|  country|   sex|nbsellers|meanproductssold|meanproductslisted|meansellerpassrate|totalproductssold|totalproductslisted|meanproductsbought|meanproductswished|meanproductsliked|totalbought|totalwished|totalproductsliked|meanfollowers|meanfollows|percentofappusers|percentofiosusers|     meanseniority|
+---------+------+---------+----------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+-----------+-----------+------------------+-------------+-----------+-----------------+-----------------+------------------+
|Allemagne|Female|      116|            4.03|              2.72|             27.33|           